In [13]:
import os

MIMIC_PATH = '/Users/azmannads/Downloads/Dr. Zhang/mimic-iv-2.2/'

files_to_check = [
    'hosp/patients.csv.gz',
    'hosp/admissions.csv.gz',
    'hosp/diagnoses_icd.csv.gz',
    'hosp/labevents.csv.gz',
    'icu/icustays.csv.gz',
    'icu/chartevents.csv.gz'
]

print("Checking files...")
for file in files_to_check:
    full_path = os.path.join(MIMIC_PATH, file)
    if os.path.exists(full_path):
        size = os.path.getsize(full_path) / (1024**3)
        print(f"✓ {file} ({size:.2f} GB)")
    else:
        print(f"✗ {file} - NOT FOUND")

Checking files...
✓ hosp/patients.csv.gz (0.00 GB)
✓ hosp/admissions.csv.gz (0.02 GB)
✓ hosp/diagnoses_icd.csv.gz (0.03 GB)
✓ hosp/labevents.csv.gz (2.41 GB)
✓ icu/icustays.csv.gz (0.00 GB)
✓ icu/chartevents.csv.gz (3.26 GB)
